# Use BrainWeb

In [1]:
import torch
import brainweb
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from __future__ import print_function, division
%matplotlib notebook
import brainweb
from brainweb import volshow
import numpy as np
from os import path
from tqdm.auto import tqdm
import logging
logging.basicConfig(level=logging.INFO)

## View data

In [4]:
%matplotlib inline
# download
files = brainweb.get_files()

# read last file
data = brainweb.load_file(files[-1])

# show last subject
print(files[-1])
volshow(data, cmaps=['gist_ncar']);

BrainWeb Subjects:   0%|          | 0/20 [00:00<?, ?file/s]

/mnt/ssd/jj/.brainweb/subject_54.bin.gz


Output()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=181, description='z', max=361), Output()), _dom_classes=('widget-interac…

In [11]:
data.shape

(362, 434, 362)

# Transform

## Convert raw image data:

Siemens Biograph mMR resolution (~2mm) & dimensions (127, 344, 344)


PET/T1/T2/uMap intensities


PET defaults to FDG intensity ratios; could use e.g. Amyloid instead


randomised structure for PET/T1/T2


t (1 + g [2 G_sigma(r) - 1]), where


r = rand(127, 344, 344) in [0, 1),


Gaussian smoothing sigma = 1,


g = 1 for PET; 0.75 for MR, and


t = the PET or MR piecewise constant phantom#

In [20]:
# show region probability masks
PetClass = brainweb.FDG
label_probs = brainweb.get_label_probabilities(files[-1], labels=PetClass.all_labels)
volshow(label_probs[brainweb.trim_zeros_ROI(label_probs)], titles=PetClass.all_labels, frameon=False);

BrainWeb labels:   0%|          | 0/12 [00:00<?, ?label/s]

Trimming ROI:   0%|          | 0/4 [00:00<?, ?it/s]

Output()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=44, description='z', max=88), Output()), _dom_classes=('widget-interact'…

In [21]:
brainweb.seed(1337)

for f in tqdm(files, desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(
        f,
        petNoise=1, t1Noise=0.75, t2Noise=0.75,
        petSigma=1, t1Sigma=1, t2Sigma=1,
        PetClass=PetClass)
        

mMR ground truths:   0%|          | 0/20 [00:00<?, ?subject/s]

In [34]:
print(vol['T1'].shape)
print(vol['T2'].shape)
print(vol['PET'].shape)

(127, 344, 344)
(127, 344, 344)
(127, 344, 344)


In [22]:
# show last subject
print(f)
volshow([vol['PET' ][:, 100:-100, 100:-100],
         vol['uMap'][:, 100:-100, 100:-100],
         vol['T1'  ][:, 100:-100, 100:-100],
         vol['T2'  ][:, 100:-100, 100:-100]],
        cmaps=['hot', 'bone', 'Greys_r', 'Greys_r'],
        titles=["PET", "uMap", "T1", "T2"],
        frameon=False);

/mnt/ssd/jj/.brainweb/subject_54.bin.gz


Output()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=63, description='z', max=126), Output()), _dom_classes=('widget-interact…

In [26]:
# add some lesions
brainweb.seed(1337)
im3d = brainweb.add_lesions(vol['T1'])
volshow(im3d[:, 100:-100, 100:-100], cmaps=['hot']);

Output()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=63, description='z', max=126), Output()), _dom_classes=('widget-interact…

### Only with T1, T2

In [25]:
# bonus: use brute-force registration to transform
#!pip install -U 'brainweb[register]'
reg = brainweb.register(
    data[:, ::-1], target=vol['PET'],
    src_resolution=brainweb.Res.brainweb,
    target_resolution=brainweb.Res.mMR)

volshow({
    "PET":    vol['PET'][:, 100:-100, 100:-100],
    "RawReg": reg[       :, 100:-100, 100:-100],
    "T1":     vol['T1' ][:, 100:-100, 100:-100],
}, cmaps=['hot', 'gist_ncar', 'Greys_r'], ncols=3, tight_layout=5, figsize=(9.5, 3.5), frameon=False);

ModuleNotFoundError: No module named 'dipy'

## Concat T1 and T2 data

In [8]:
%matplotlib inline
# download
files = brainweb.get_files()

# read last file
data = brainweb.load_file(files[-1])

for f in tqdm(files, desc="mMR ground truths", unit="subject"):
    print(f)


BrainWeb Subjects:   0%|          | 0/20 [00:00<?, ?file/s]

mMR ground truths:   0%|          | 0/20 [00:00<?, ?subject/s]

/mnt/ssd/jj/.brainweb/subject_04.bin.gz
/mnt/ssd/jj/.brainweb/subject_05.bin.gz
/mnt/ssd/jj/.brainweb/subject_06.bin.gz
/mnt/ssd/jj/.brainweb/subject_18.bin.gz
/mnt/ssd/jj/.brainweb/subject_20.bin.gz
/mnt/ssd/jj/.brainweb/subject_38.bin.gz
/mnt/ssd/jj/.brainweb/subject_41.bin.gz
/mnt/ssd/jj/.brainweb/subject_42.bin.gz
/mnt/ssd/jj/.brainweb/subject_43.bin.gz
/mnt/ssd/jj/.brainweb/subject_44.bin.gz
/mnt/ssd/jj/.brainweb/subject_45.bin.gz
/mnt/ssd/jj/.brainweb/subject_46.bin.gz
/mnt/ssd/jj/.brainweb/subject_47.bin.gz
/mnt/ssd/jj/.brainweb/subject_48.bin.gz
/mnt/ssd/jj/.brainweb/subject_49.bin.gz
/mnt/ssd/jj/.brainweb/subject_50.bin.gz
/mnt/ssd/jj/.brainweb/subject_51.bin.gz
/mnt/ssd/jj/.brainweb/subject_52.bin.gz
/mnt/ssd/jj/.brainweb/subject_53.bin.gz
/mnt/ssd/jj/.brainweb/subject_54.bin.gz


In [9]:
brainweb.seed(1337)
stack = None


for f in tqdm(files, desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(
        f,
        petNoise=1, t1Noise=0.75, t2Noise=0.75,
        petSigma=1, t1Sigma=1, t2Sigma=1,)

    t1=vol['T1'][:, 100:-100, 100:-100]
    t2=vol['T2'][:, 100:-100, 100:-100]

    if stack is None:
        stack = np.concatenate((np.expand_dims(t1, 0), np.expand_dims(t2, 0)), axis=1)

    else:
        stack = np.concatenate((stack, np.concatenate((np.expand_dims(t1, 0), np.expand_dims(t2, 0)), axis=1)))
    print(stack.shape)


mMR ground truths:   0%|          | 0/20 [00:00<?, ?subject/s]

/mnt/ssd/jj/.brainweb/subject_04.bin.gz
(1, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_05.bin.gz
(2, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_06.bin.gz
(3, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_18.bin.gz
(4, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_20.bin.gz
(5, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_38.bin.gz
(6, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_41.bin.gz
(7, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_42.bin.gz
(8, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_43.bin.gz
(9, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_44.bin.gz
(10, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_45.bin.gz
(11, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_46.bin.gz
(12, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_47.bin.gz
(13, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_48.bin.gz
(14, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_49.bin.gz
(15, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_50.bin.gz
(16, 254, 144, 144)
/mnt/ssd/jj/.brainweb/subject_51.bin.gz
(17, 254,

### Why cut??

In [12]:
print(vol['T1'].shape)
print(vol['T1'][:, 100:-100, 100:-100].shape)
volshow(vol['T1'][:, 100:-100, 100:-100], cmaps=['hot'])
volshow(vol['T1'], cmaps=['hot'])

Output()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=63, description='z', max=126), Output()), _dom_classes=('widget-interact…

Output()

<Figure size 432x288 with 0 Axes>

interactive(children=(IntSlider(value=63, description='z', max=126), Output()), _dom_classes=('widget-interact…

<function brainweb.utils.volshow.<locals>.plot_slice(z)>

(127, 144, 144)